# Time Series - Salinity (and a little bit about precipitation) 

## Load the necessary modules and the time series file:

In [47]:
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import matplotlib.dates as dates
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import statsmodels.api as sm
import datetime
%matplotlib qt5
plt.rcParams.update({'font.size': 18,'font.family':'serif','font.serif':'Arial'})

## Load data

We will use pandas to load a csv file:
We add one file with temperature, salinity, etc. And other dataset with precipitation

In [48]:

data=pd.read_csv('C:/Users/brdera001/Downloads/Time_series/Python/data.csv')
data['Date'] = pd.to_datetime(data['Date'])
prec = pd.read_csv('C:/Users/brdera001/Downloads/Time_series/Python/precipitation_helgoland.csv')
prec['Date'] = pd.to_datetime(prec['Date'])


In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21275 entries, 0 to 21274
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         21275 non-null  datetime64[ns]
 1   SECCHI       11917 non-null  float64       
 2   temperature  13864 non-null  float64       
 3   Salinity     13669 non-null  float64       
 4   Chl_a_Hplc   1585 non-null   float64       
 5   pH           2018 non-null   float64       
 6   sunlight     19084 non-null  float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 1.1 MB


In [50]:
prec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23191 entries, 0 to 23190
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           23191 non-null  datetime64[ns]
 1   precipitation  23191 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 362.5 KB


In [51]:
#merge the datasets
data_prec = pd.merge(data, prec, on = "Date")
data_prec.describe()

,Date,SECCHI,temperature,Salinity,Chl_a_Hplc,pH,sunlight,precipitation
count,21275,11917.000000,13864.000000,13669.000000,1585.000000,2018.000000,19084.000000,21275.000000
mean,1991-02-15 00:00:00,3.571159,10.279645,32.147013,1.938774,8.217589,4.766438,2.024761
min,1962-01-01 00:00:00,0.100000,-1.600000,22.745000,0.027000,7.140000,0.000000,0.000000
25%,1976-07-24 12:00:00,2.000000,5.700000,31.519000,0.624000,8.154000,0.400000,0.000000
50%,1991-02-15 00:00:00,3.500000,10.200000,32.332000,1.196000,8.200000,3.700000,0.100000
75%,2005-09-07 12:00:00,4.900000,15.100000,33.014000,2.208000,8.270000,8.100000,2.200000
max,2020-03-31 00:00:00,12.500000,20.200000,36.106000,70.600000,9.570000,16.600000,64.900000
std,NaN,1.833159,5.041619,1.220393,3.090959,0.121145,4.527239,4.151484


In [6]:
#chech for duplicates
data_prec[data_prec.duplicated(subset=['Date'])]

,Date,SECCHI,temperature,Salinity,Chl_a_Hplc,pH,sunlight,precipitation


In [202]:
#GENERAL Correlation plot
sns.heatmap(data_prec_n.corr())

<Axes: >

In [10]:
#remove nan para o heatmap
data_prec_n=data_prec.dropna()

In [28]:
#heatmap better 
plt.figure(figsize=(12, 12))  # Increase the figure size
corr_matrix=data_prec_n.corr()
mask = np.zeros_like(corr_matrix)
mask[np.triu_indices_from(mask)] = True

# Create the heatmap using seaborn
sns.heatmap(corr_matrix, mask=mask, cmap='Spectral', annot=True, fmt=".2f", linewidths=0.5)

# Rotate the x-axis labels to vertical orientation
plt.xticks(rotation=90)

# Rotate the y-axis labels to horizontal orientation
plt.xticks(rotation=360)

# Increase font size for x and y axis labels
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.title('Correlation Matrix Heatmap')
plt.show()

## Boxplot - Salinity and precipitation

In [52]:
# Boxplot salinity

plt.figure(figsize=(14,6),constrained_layout=True)
ax = sns.boxplot(x=data.Date.dt.month, y="Salinity", data=data_prec,
                 showmeans=True,showfliers = True,color='darkgrey',#palette='colorblind',
                 meanprops={'marker': 'o','markeredgecolor': 'black','markerfacecolor':'black','markersize':'5'},
                )
ax.set_ylabel('Salinity')
ax.grid(ls='--')


In [53]:
# Boxplot precipitation
plt.figure(figsize=(14,6),constrained_layout=True)
ax = sns.boxplot(x=data.Date.dt.month, y="precipitation", data=data_prec,
                 showmeans=True,showfliers = True,color='darkgrey',#palette='colorblind',
                 meanprops={'marker': 'o','markeredgecolor': 'black','markerfacecolor':'black','markersize':'5'},
                )
ax.set_ylabel('Precipitation (mm)')
ax.grid(ls='--')

Boxplot defines outliers as follow:

below Q1 − 1.5 IQR 
or 
above Q3 + 1.5 IQR

Q1 first quartile (25%)

Q3 third quartile (75%)

IQR = Q3 - Q1

below Q1 − 1.5 IQR or above Q3 + 1.5 IQR #just the formula

In [54]:
#Salinity outliers
q1=data_prec.Salinity[data_prec.Date.dt.month==1].describe()['25%']
q3=data_prec.Salinity[data_prec.Date.dt.month==1].describe()['75%']
iqr=q3-q1

In [55]:
#precipitation outliars
q1=data_prec.precipitation[data_prec.Date.dt.month==1].describe()['25%']
q3=data_prec.precipitation[data_prec.Date.dt.month==1].describe()['75%']
iqr=q3-q1

In [35]:
iqr

2.4

In [56]:
#create a copy
data_c=data_prec.copy()

In [57]:
#for salinity outliars
months=np.arange(1,13)
for ii in months:

    q1=data_prec.Salinity[data_prec.Date.dt.month==ii].describe()['25%']
    q3=data_prec.Salinity[data_prec.Date.dt.month==ii].describe()['75%']
    iqr=q3-q1

    filter = (data_prec['Salinity'][data_prec.Date.dt.month==ii] >= q1 - 1.5 * iqr) & (data_prec['Salinity'][data_prec.Date.dt.month==ii] <= q3 + 1.5 *iqr)
    data_c.Salinity[data_c.Date.dt.month==ii]=data_c.Salinity[data_c.Date.dt.month==ii][filter]

C:\Users\brdera001\AppData\Local\Temp\ipykernel_18660\3765709812.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_c.Salinity[data_c.Date.dt.month==ii]=data_c.Salinity[data_c.Date.dt.month==ii][filter]
C:\Users\brdera001\AppData\Local\Temp\ipykernel_18660\3765709812.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_c.Salinity[data_c.Date.dt.month==ii]=data_c.Salinity[data_c.Date.dt.month==ii][filter]
C:\Users\brdera001\AppData\Local\Temp\ipykernel_18660\3765709812.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http

In [58]:
#for PRECIPITATION outliars
months=np.arange(1,13)
for ii in months:

    q1=data_prec.precipitation[data_prec.Date.dt.month==ii].describe()['25%']
    q3=data_prec.precipitation[data_prec.Date.dt.month==ii].describe()['75%']
    iqr=q3-q1

    filter = (data_prec['precipitation'][data_prec.Date.dt.month==ii] >= q1 - 1.5 * iqr) & (data_prec['precipitation'][data_prec.Date.dt.month==ii] <= q3 + 1.5 *iqr)
    data_c.precipitation[data_c.Date.dt.month==ii]=data_c.precipitation[data_c.Date.dt.month==ii][filter]

C:\Users\brdera001\AppData\Local\Temp\ipykernel_18660\623218170.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_c.precipitation[data_c.Date.dt.month==ii]=data_c.precipitation[data_c.Date.dt.month==ii][filter]
C:\Users\brdera001\AppData\Local\Temp\ipykernel_18660\623218170.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_c.precipitation[data_c.Date.dt.month==ii]=data_c.precipitation[data_c.Date.dt.month==ii][filter]
C:\Users\brdera001\AppData\Local\Temp\ipykernel_18660\623218170.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the do

In [95]:
# Boxplot without outliers Salinity

plt.figure(figsize=(14,6),constrained_layout=True)
ax = sns.boxplot(x=data_c.Date.dt.month, y="Salinity", data=data_c,
                 showmeans=True,showfliers = True,color='darkgrey',#palette='colorblind',
                 meanprops={'marker': 'o','markeredgecolor': 'black','markerfacecolor':'black','markersize':'5'},
                )
ax.set_ylabel('Salinity')
ax.grid(ls='--')

In [94]:
# Boxplot without outliers precipitation

plt.figure(figsize=(14,6),constrained_layout=True)
ax = sns.boxplot(x=data_c.Date.dt.month, y="precipitation", data=data_c,
                 showmeans=True,showfliers = True,color='darkgrey',#palette='colorblind',
                 meanprops={'marker': 'o','markeredgecolor': 'black','markerfacecolor':'black','markersize':'5'},
                )
ax.set_ylabel('precipitation (mm)')
ax.grid(ls='--')

## Removing by standard deviation

In a normal distribution, the standard deviation is as follows:

1 Standard Deviation from the Mean: 68%

2 Standard Deviations from the Mean: 95%

3 Standard Deviations from the Mean: 99.7%


In [61]:
data2=data_prec.copy()

In [62]:
#identify and remove outliers for prec
months=np.arange(1,13)
for ii in months:

    data_std=data2.precipitation[data2.Date.dt.month==ii].describe()['std']
    data_mean=data2.precipitation[data2.Date.dt.month==ii].describe()['mean']    

    filter = (data2['precipitation'][data2.Date.dt.month==ii] >= data_mean-data_std*3) & (data2['precipitation'][data2.Date.dt.month==ii] <= data_mean+data_std*3)
    data2.precipitation[data2.Date.dt.month==ii]=data2.precipitation[data2.Date.dt.month==ii][filter]

C:\Users\brdera001\AppData\Local\Temp\ipykernel_18660\399798275.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2.precipitation[data2.Date.dt.month==ii]=data2.precipitation[data2.Date.dt.month==ii][filter]
C:\Users\brdera001\AppData\Local\Temp\ipykernel_18660\399798275.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2.precipitation[data2.Date.dt.month==ii]=data2.precipitation[data2.Date.dt.month==ii][filter]
C:\Users\brdera001\AppData\Local\Temp\ipykernel_18660\399798275.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

In [65]:
plt.figure(figsize=(12,6),constrained_layout=True)
plt.plot(data_prec.Date,data_prec.precipitation,'r')
months=np.arange(1,13)
for ii in months:

    data_std=data_prec.precipitation[data_prec.Date.dt.month==ii].describe()['std']
    data_mean=data_prec.precipitation[data_prec.Date.dt.month==ii].describe()['mean'] 
    plt.fill_between(x=data_prec.Date[data_prec.Date.dt.month==ii], y1=data_mean-data_std*3,
                     y2=data_mean+data_std*3,color='lightgrey',alpha=0.5)  
plt.ylabel('Precipitation (mm)')
plt.xlim(data_prec.Date[0],data_prec.Date.iloc[-1])
plt.grid(ls='--')


In [66]:
# Boxplot by year

plt.figure(figsize=(14,6),constrained_layout=True)
bp = sns.boxplot(x=data_prec.Date.dt.year, y="precipitation", data=data_prec,
                 showmeans=True,showfliers = True,color='darkgrey',#palette='colorblind',
                 meanprops={'marker': 'o','markeredgecolor': 'black','markerfacecolor':'black','markersize':'5'},
                )
bp.set_ylabel('Precipitation (mm)')
bp.set_xticklabels(bp.get_xticklabels(),rotation=60)
bp.grid(ls='--')

## Very important, always plot your data before start analysis

In [71]:
plt.figure(constrained_layout=True)
plt.plot(data_prec.Date,data_prec.Salinity,'r',label='Salinity')
plt.grid(ls='--')
plt.legend()

In [70]:
plt.figure(constrained_layout=True)
plt.plot(data_prec.Date,data_prec.precipitation,'r',label='Precipitation (mm)')
plt.grid(ls='--')
plt.legend()

In [73]:
#salinity plot
plt.figure(figsize=(12,6),constrained_layout=True)
plt.plot(data_prec.Date,data_c.Salinity,'-r',label='Salinity') #here using the clean data (data_c)
plt.xlim(data_prec.Date[0],data_prec.Date.iloc[-1])
plt.xlabel('Date')
plt.ylabel('Salinity')
plt.legend()
plt.grid(ls='--')

In [74]:
#precipitation plot
plt.figure(figsize=(12,6),constrained_layout=True)
plt.plot(data_prec.Date,data_c.precipitation,'-r',label='precipitation') #here using the clean data (data_c)
plt.xlim(data_prec.Date[0],data_prec.Date.iloc[-1])
plt.xlabel('Date')
plt.ylabel('Precipitation (mm)')
plt.legend()
plt.grid(ls='--')

In [76]:
#SALINITY AND PRECIPITATION
plt.figure(figsize=(12,6),constrained_layout=True)
pc1=plt.plot(data_prec.Date,data_c.precipitation,'-y',label='Precipitation') #here using the clean data (data_c)
pc2=plt.plot(data_prec.Date,data_c.Salinity,'-r',label='Salinity') #here using the clean data (data_c)
plt.xlim(data_prec.Date[0],data_prec.Date.iloc[-1])
plt.xlabel('Date')
plt.ylabel('Precipitation (mm)\n Salinity')
plt.legend()
plt.grid(ls='--')

## Subplot

In [77]:
# Create figure and axis objects with subplots()
fig,axs = plt.subplots(2, 1, figsize=(14,7),constrained_layout=True,sharex=True)
axs=axs.flatten()

# Make a plot
pp1,=axs[0].plot(data_prec.Date,data_c.Salinity,'b-.',label='Salinity') #here using the clean data (data_c)
axs[0].set_ylabel('Salinity')
axs[0].set_xlim(data_prec.Date[0],data_prec.Date.iloc[-1])
axs[0].grid(ls='dotted')
axs[0].legend()

pp2,=axs[1].plot(data_prec.Date,data_c.precipitation,'g',label='Precipitation') #here using the clean data (data_c)
axs[1].set_ylabel('Precipitation (mm)')
axs[1].legend()
axs[1].set_xlim(data_prec.Date[0],data_prec.Date.iloc[-1])
axs[1].grid(ls='dotted')

## Plot two variables with two y-axis

In [78]:
# create figure and axis objects with subplots()
fig,ax = plt.subplots(figsize=(12,6),constrained_layout=True,sharex=True)
# make a plot
pc1,=ax.plot(data_prec.Date,data_c.precipitation,'g') #here using the clean data (data_c)
# set y-axis label
ax.set_ylabel('Precipitation (mm)',color='g')
ax.grid(ls='--')
twin1=ax.twinx()
# make a plot with different y-axis using second axis object
pc2,=twin1.plot(data_prec.Date,data_c.Salinity,'b') #here using the clean data (data_c)
twin1.set_ylabel('Salinity',color='b')
twin1.legend([pc1,pc2],['Precipitation','Salinity'])
twin1.set_xlim(data.Date[0],data.Date.iloc[-1])
twin1.grid(ls='dotted')

In [80]:
yearlist = np.unique(data_prec.Date.dt.year)

## Shading a period

In [83]:
plt.figure(figsize=(12,6))
sc=plt.plot(data_prec.Date,data_c.Salinity,'b',label='Salinity') #here using the clean data (data_c)

for ii in range(len(yearlist)):
        
    sc1=plt.axvspan(dates.date2num(datetime.datetime(int(yearlist[ii]),4,1)), dates.date2num(datetime.datetime(int(yearlist[ii]),6,1))
                           , color='red', alpha=0.25)
    sc2=plt.axvspan(dates.date2num(datetime.datetime(int(yearlist[ii]),9,1)), dates.date2num(datetime.datetime(int(yearlist[ii]),11,1))
                           , color='yellow', alpha=0.25)
plt.ylabel('Salinity')    
plt.legend([sc1,sc2],['spring','autumn'])
plt.grid(ls='--')

## Groupby with aggregate function

In [84]:
def q25(x):
    return x.quantile(0.25)
def q75(x):
    return x.quantile(0.75)
def p5(x):
    return x.quantile(0.05)
def p95(x):
    return x.quantile(0.95)

In [92]:
data_month=data_c.drop('Date',axis=1).groupby(data_c.Date.dt.month).agg(
    ["mean","median","std","max","min","count",q25,q75,p5,p95]).rename_axis(
    ['month']).copy()
#here using the clean data (data_c)

In [93]:
data_month.precipitation

,mean,median,std,max,min,count,q25,q75,p5,p95
month,,,,,,,,,,
1,1.013083,0.2,1.498957,6.0,0.0,1651,0.0,1.600,0.0,4.50
2,0.524190,0.0,0.906445,3.7,0.0,1451,0.0,0.700,0.0,2.70
3,0.529057,0.0,0.916645,3.7,0.0,1590,0.0,0.700,0.0,2.90
4,0.360120,0.0,0.688207,2.8,0.0,1497,0.0,0.400,0.0,2.20
5,0.295825,0.0,0.619694,2.7,0.0,1509,0.0,0.200,0.0,1.90
6,0.433266,0.0,0.850339,3.7,0.0,1476,0.0,0.400,0.0,2.60
7,0.648678,0.0,1.203467,5.2,0.0,1551,0.0,0.700,0.0,3.65
8,0.751323,0.0,1.356102,6.0,0.0,1549,0.0,0.900,0.0,4.10
9,1.093811,0.0,1.854430,7.6,0.0,1535,0.0,1.400,0.0,5.50


In [99]:
#salinity plot with cleaned data
plt.figure(figsize=(10,8),constrained_layout=True)
plt.plot(data_month.index,data_month.Salinity['median'],'k',linewidth=2)
plt.fill_between(data_month.index,data_month.Salinity['q25'],data_month.Salinity['q75'],color='b',alpha=0.2 )
plt.fill_between(data_month.index,data_month.Salinity['p5'],data_month.Salinity['p95'],color='b',alpha=0.05 )
for ii in data_month.index:
    sc=plt.scatter(np.repeat(ii, len(data_c.Salinity[data_c.Date.dt.month==ii])),
                   data_c.Salinity[data_c.Date.dt.month==ii],c=data_c.Salinity[data_c.Date.dt.month==ii],cmap='viridis')
cb=plt.colorbar(sc,shrink=0.6,aspect=10)
cb.ax.set_title('Salinity')
plt.xticks(data_month.index)
plt.xlabel('Months')
plt.ylabel('Salinity')
plt.grid(ls='--')

In [116]:
#precipitation plot with cleaned data
plt.figure(figsize=(10,8),constrained_layout=True)
plt.plot(data_month.index,data_month.precipitation['median'],'k',linewidth=2)
plt.fill_between(data_month.index,data_month.precipitation['q25'],data_month.precipitation['q75'],color='grey',alpha=0.2 )
plt.fill_between(data_month.index,data_month.precipitation['p5'],data_month.precipitation['p95'],color='grey',alpha=0.1 )
for ii in data_month.index:
    sc=plt.scatter(np.repeat(ii, len(data_c.precipitation[data.Date.dt.month==ii])),
                   data_c.precipitation[data_c.Date.dt.month==ii],c=data_c.precipitation[data_c.Date.dt.month==ii],cmap='Blues')
cb=plt.colorbar(sc,shrink=0.6,aspect=10)
cb.ax.set_title('(mm)')
plt.xticks(data_month.index)
plt.xlabel('Months')
plt.ylabel('Precipitation(mm)')
plt.grid(ls='--')

## Calculate Anomalies

In [117]:
#create the dataset to input anomalies data
sal_anom=data_c[['Date','Salinity']].copy()
prec_anom=data_c[['Date','precipitation']].copy()

In [118]:
sal_anom

,Date,Salinity
0,1962-01-01,NaN
1,1962-01-02,NaN
2,1962-01-03,NaN
3,1962-01-04,NaN
4,1962-01-05,NaN
...,...,...
21270,2020-03-27,31.677
21271,2020-03-28,NaN
21272,2020-03-29,NaN
21273,2020-03-30,31.568


In [119]:
#calculation of anomalies
months=np.arange(1,13,1)

for month in months: 

    sal_anom.loc[sal_anom['Date'].dt.month==month,'anomaly']=(sal_anom['Salinity'])-data_month.Salinity.loc[month,'mean']
    prec_anom.loc[prec_anom['Date'].dt.month==month,'anomaly']=(prec_anom['precipitation'])-data_month.precipitation.loc[month,'mean']

In [120]:
#plot salinity anomaly and the salinity data
plt.figure(figsize=(12,6),constrained_layout=True)
plt.plot(sal_anom.Date,sal_anom.Salinity,'r')
plt.plot(sal_anom.Date,sal_anom.anomaly,'k')
plt.xlabel(None)
plt.ylabel('Salinity')
plt.axhline(0, color='k')
plt.grid(ls='--')
## here I'm adding a column with colors
#color_ts = np.where(temp_anom.anomaly<0, 'red', 'blue')

In [122]:
#plot precipiation anomaly and the salinity data
plt.figure(figsize=(12,6),constrained_layout=True)
plt.plot(prec_anom.Date,prec_anom.precipitation,'r')
plt.plot(prec_anom.Date,prec_anom.anomaly,'k')
plt.xlabel(None)
plt.ylabel('Precipitation (mm)')
plt.axhline(0, color='k')
plt.grid(ls='--')
## here I'm adding a column with colors
#color_ts = np.where(temp_anom.anomaly<0, 'red', 'blue')

## The seasonal signal

In [123]:
sal_month=sal_anom.set_index('Date').resample('M').mean()
sal_month.reset_index(inplace=True,drop=False)

In [124]:
plt.figure(figsize=(12,6),constrained_layout=True)
plt.plot(sal_month.Date,sal_month.Salinity-sal_month.anomaly,'o-')
plt.ylabel('Salinity')
plt.xlim(data_prec.Date[0],data_prec.Date.iloc[-1])
plt.grid(ls='--')

In [125]:
plt.figure(figsize=(12,6),constrained_layout=True)
plt.fill_between(x=sal_anom.Date, y1=sal_anom.anomaly, y2=0, where=sal_anom.anomaly >=0, interpolate=True,color='red')
plt.fill_between(x=sal_anom.Date, y1=sal_anom.anomaly, y2=0, where=sal_anom.anomaly <0, interpolate=True,color='blue')
plt.axhline(0, color='k',lw=0.5)
plt.ylabel('Salinity')
plt.xlim(data.Date[0],data.Date.iloc[-1])
plt.grid(ls='--')

## Resample

In [128]:
sal_year=sal_anom.set_index('Date').resample('Y').mean()
prec_year=prec_anom.set_index('Date').resample('Y').mean()

In [129]:
prec_year

,precipitation,anomaly
Date,,
1962-12-31,0.840491,-0.034296
1963-12-31,0.764087,-0.107217
1964-12-31,0.864412,-0.006038
1965-12-31,0.898706,0.013884
1966-12-31,0.996091,0.119195
1967-12-31,1.094357,0.208853
1968-12-31,0.805231,-0.065137
1969-12-31,0.779331,-0.088715
1970-12-31,0.961270,0.087456


In [130]:
sal_year.reset_index(inplace=True,drop=False)
prec_year.reset_index(inplace=True,drop=False)

In [131]:
#yearly salinity anomalies
plt.figure(figsize=(10,6),constrained_layout=True)
plt.bar(sal_year.Date[sal_year.anomaly>=0],sal_year.anomaly[sal_year.anomaly>=0],width=300, color='r')
plt.bar(sal_year.Date[sal_year.anomaly<0],sal_year.anomaly[sal_year.anomaly<0],width=300, color='b')
plt.axhline(0,color='k',lw=0.5)
#plt.xlim(data.Date[0],data.Date.iloc[-1])
plt.grid(ls='dotted')
plt.ylabel('Salnity anomalies')
plt.xlabel('Year')

Text(0.5, 0, 'Year')

In [133]:
#yearly prec anomalies
plt.figure(figsize=(10,6),constrained_layout=True)
plt.bar(sal_year.Date[prec_year.anomaly>=0],prec_year.anomaly[prec_year.anomaly>=0],width=300, color='r')
plt.bar(sal_year.Date[prec_year.anomaly<0],prec_year.anomaly[prec_year.anomaly<0],width=300, color='b')
plt.axhline(0,color='k',lw=0.5)
#plt.xlim(data.Date[0],data.Date.iloc[-1])
plt.grid(ls='dotted')
plt.ylabel(' Precipitation anomalies')
plt.xlabel('Year')

Text(0.5, 0, 'Year')

In [49]:
#fill between bars
plt.figure(figsize=(12,6),constrained_layout=True)
plt.fill_between(x=temp_year.Date, y1=temp_year.anomaly, y2=0, where=temp_year.anomaly >=0, interpolate=True,color='red')
plt.fill_between(x=temp_year.Date, y1=temp_year.anomaly, y2=0, where=temp_year.anomaly <0, interpolate=True,color='blue')
plt.axhline(0, color='k',lw=0.5)
plt.ylabel('Temperature anomalies (°)')
plt.grid(ls='--')

In [135]:
sal_month=sal_anom.set_index('Date').resample('M').mean()
sal_month.reset_index(inplace=True,drop=False)
sal_month

,Date,Salinity,anomaly
0,1962-01-31,32.732750,-0.032404
1,1962-02-28,30.983000,-1.604593
2,1962-03-31,30.666308,-1.723868
3,1962-04-30,30.938444,-0.776758
4,1962-05-31,30.416706,-1.300438
...,...,...,...
694,2019-11-30,33.264625,0.480932
695,2019-12-31,33.451588,0.576039
696,2020-01-31,33.542000,0.776846
697,2020-02-29,32.415667,-0.171926


In [149]:
prec_month=prec_anom.set_index('Date').resample('M').mean()
prec_month.reset_index(inplace=True,drop=False)
prec_month

,Date,precipitation,anomaly
0,1962-01-31,1.459259,0.446176
1,1962-02-28,0.576000,0.051810
2,1962-03-31,0.613793,0.084736
3,1962-04-30,0.320833,-0.039287
4,1962-05-31,0.434615,0.138790
...,...,...,...
694,2019-11-30,2.303704,0.366865
695,2019-12-31,1.355172,-0.094918
696,2020-01-31,1.288000,0.274917
697,2020-02-29,1.255556,0.731365


In [140]:
plt.figure(figsize=(12,6),constrained_layout=True)
plt.fill_between(x=temp_month.Date, y1=temp_month.anomaly, y2=0, where=temp_month.anomaly >=0, 
                 interpolate=True,color='red',label='positive')
plt.fill_between(x=temp_month.Date, y1=temp_month.anomaly, y2=0, where=temp_month.anomaly <0, 
                 interpolate=True,color='blue',label='negative')
plt.axhline(0, color='k',lw=0.5)
plt.ylabel('Temperature anomalies (°)')
plt.legend(loc='upper left')
plt.grid(ls='--')

NameError: name 'temp_month' is not defined

## Imputation

Fill missing values using statistics

In [93]:
## Filling using mean or median

tmp=data[['Date','temperature']]

tmp = tmp.assign(FillMean=tmp.temperature.fillna(tmp.temperature.mean()))
tmp = tmp.assign(FillMedian=tmp.temperature.fillna(tmp.temperature.median()))

tmp

,Date,temperature,FillMean,FillMedian
0,1962-01-01,NaN,10.279645,10.2
1,1962-01-02,4.6,4.600000,4.6
2,1962-01-03,5.1,5.100000,5.1
3,1962-01-04,4.3,4.300000,4.3
4,1962-01-05,5.1,5.100000,5.1
...,...,...,...,...
21270,2020-03-27,6.6,6.600000,6.6
21271,2020-03-28,NaN,10.279645,10.2
21272,2020-03-29,NaN,10.279645,10.2
21273,2020-03-30,6.5,6.500000,6.5


In [97]:
plt.figure(figsize=(12,6),constrained_layout=True)
plt.plot(tmp.Date,tmp.temperature,'sk',label='original')
plt.plot(tmp.Date,tmp.FillMean,'*r',label='mean')
plt.ylabel('Temperature (°)')
plt.grid(ls='--')
plt.legend()

In [98]:
plt.figure(figsize=(12,6),constrained_layout=True)
plt.plot(tmp.Date,tmp.temperature,'sk',label='original')
plt.plot(tmp.Date,tmp.FillMedian,'*r',label='median')
plt.ylabel('Temperature (°)')
plt.grid(ls='--')
plt.legend()

## Filling using the rolling median

In [99]:
tmp['RollingMean']= tmp.temperature.fillna(tmp.temperature.rolling(10,center=True,min_periods=1,).mean())

tmp = tmp.assign(RollingMedian=tmp.temperature.fillna(tmp.temperature.rolling(10,center=True,min_periods=1,)
                                                                .median()))

In [106]:
tmp.RollingMean[tmp.RollingMean.isnull()]

Series([], Name: RollingMean, dtype: float64)

In [105]:
tmp.RollingMedian[tmp.RollingMedian.isnull()]

Series([], Name: RollingMedian, dtype: float64)

In [102]:
plt.figure(figsize=(12,6),constrained_layout=True)
plt.plot(tmp.Date,tmp.temperature,'sk',label='original')
plt.plot(tmp.Date,tmp.RollingMean,'*r',label='rolling mean')
plt.ylabel('Temperature (°)')
plt.grid(ls='--')
plt.legend()

In [108]:
plt.figure(figsize=(12,6),constrained_layout=True)
plt.plot(tmp.Date,tmp.temperature,'sk',label='original')
plt.plot(tmp.Date,tmp.RollingMedian,'*r',label='rolling median')
plt.ylabel('Temperature (°)')
plt.grid(ls='--')
plt.legend()


In [ ]:
from sklearn.impute import KNNImputer

tmp = tmp.assign(InterpolateLinear=tmp.temperature.interpolate(method='linear'))
tmp = tmp.assign(InterpolateQuadratic=tmp.temperature.interpolate(method='quadratic'))
tmp = tmp.assign(InterpolateCubic=tmp.temperature.interpolate(method='cubic'))
tmp = tmp.assign(InterpolateSLinear=tmp.temperature.interpolate(method='slinear')) # spline linear
tmp = tmp.assign(InterpolateAkima=tmp.temperature.interpolate(method='akima'))
tmp = tmp.assign(InterpolatePoly3=tmp.temperature.interpolate(method='polynomial', order=3))
tmp = tmp.assign(InterpolatePoly5=tmp.temperature.interpolate(method='polynomial', order=5)) 
tmp = tmp.assign(InterpolatePoly7=tmp.temperature.interpolate(method='polynomial', order=7))
tmp = tmp.assign(InterpolateSpline3=tmp.temperature.interpolate(method='spline', order=3))
tmp = tmp.assign(InterpolateSpline4=tmp.temperature.interpolate(method='spline', order=4))
tmp = tmp.assign(InterpolateSpline5=tmp.temperature.interpolate(method='spline', order=5))
tmp = tmp.assign(ffill=tmp.temperature.ffill())
tmp = tmp.assign(bfill = tmp.temperature.bfill())
imputer = KNNImputer(n_neighbors=5)
tmp = tmp.assign(kNN=imputer.fit_transform(tmp['temperature'].to_frame()))


## Calculate linear trends

In [144]:
slope, intercept, r_value, p_value, std_err = sp.stats.linregress(sal_month.index,sal_month.anomaly)

In [150]:
slope, intercept, r_value, p_value, std_err = sp.stats.linregress(prec_month.index,prec_month.anomaly)

In [145]:
m, b = np.polyfit(sal_month.index,sal_month.anomaly, 1)

In [151]:
m, b = np.polyfit(prec_month.index,prec_month.anomaly, 1)

In [124]:
intercept

-0.9716899814019064

In [125]:
b

-0.9716899814019062

In [158]:
#PLOT TREND SALINITY
plt.figure(figsize=(10,6),constrained_layout=True)
plt.plot(sal_month.Date,sal_month.anomaly,'k',label='SST anomaly')
plt.plot(sal_month.Date, slope*sal_month.index+intercept,'--', color='black',label='Linear regression')
plt.text(sal_month.Date[0], 2.3, r'Trend = '+ str(round((slope*sal_month.index[-1]+intercept)-
                                                       (slope*sal_month.index[0]+intercept),2))+'/58 years')
plt.text(sal_month.Date[0], 1.7, r'Trend = '+ str(round(((slope*sal_month.index[-1]+intercept)-
                                                          (slope*sal_month.index[0]+intercept)/58),2))+'/years')
plt.ylabel('Salinity')
plt.grid(ls='--')
plt.legend(loc='lower right')

In [161]:
#PLOT TREND PREC
plt.figure(figsize=(12,6),constrained_layout=True)
plt.plot(prec_month.Date,prec_month.anomaly,'k',label='Precipitation anomaly')
plt.plot(prec_month.Date, slope*prec_month.index+intercept,'--', color='black',label='Linear regression')
plt.text(prec_month.Date[0], -1.6, r'Trend = '+ str(round((slope*prec_month.index[-1]+intercept)-
                                                       (slope*prec_month.index[0]+intercept),2))+'mm/58 years')
plt.text(prec_month.Date[0], -1.2, r'Trend = '+ str(round(((slope*prec_month.index[-1]+intercept)-
                                                          (slope*prec_month.index[0]+intercept)/58),2))+'mm/years')
plt.ylabel('Precipitation')
plt.grid(ls='--')
plt.legend(loc='lower right')

## Correlation

We will correlate SST with sunlight duration

First, we merge the two variables using date. They will have the same size and same dates.

In [100]:
sal_prec=sal_anom.merge(prec_anom,on='Date')
sal_prec

,Date,Salinity,precipitation
0,1962-01-01,NaN,0.3
1,1962-01-02,NaN,0.2
2,1962-01-03,NaN,0.3
3,1962-01-04,NaN,0.6
4,1962-01-05,NaN,1.6
...,...,...,...
21270,2020-03-27,31.677,0.0
21271,2020-03-28,NaN,0.0
21272,2020-03-29,NaN,0.4
21273,2020-03-30,31.568,1.5


In [101]:
sal_prec_no_nan=sal_prec.dropna().reset_index(drop=True)

In [102]:
sal_prec_no_nan

,Date,Salinity,precipitation
0,1962-01-15,33.480,0.0
1,1962-01-17,33.554,2.0
2,1962-01-19,33.560,4.9
3,1962-01-22,33.446,5.9
4,1962-01-24,32.861,11.0
...,...,...,...
13664,2020-03-25,33.426,0.0
13665,2020-03-26,33.828,0.0
13666,2020-03-27,31.677,0.0
13667,2020-03-30,31.568,1.5


In [103]:
sal_prec_no_nan.rename(columns={'anomaly_x':'sal_anom','anomaly_y':'prec_anom'},inplace=True)

In [104]:
sal_prec_month=sal_prec_no_nan.set_index('Date').resample('M').mean()
sal_prec_month.reset_index(inplace=True,drop=False)

In [105]:
sal_prec_month

,Date,Salinity,precipitation
0,1962-01-31,32.732750,3.512500
1,1962-02-28,30.751333,0.855556
2,1962-03-31,30.666308,1.500000
3,1962-04-30,30.938444,1.055556
4,1962-05-31,30.262889,1.444444
...,...,...,...
694,2019-11-30,33.264625,3.156250
695,2019-12-31,33.451588,1.164706
696,2020-01-31,33.542000,3.745455
697,2020-02-29,32.415667,4.522222


In [107]:
plt.figure(figsize=(10,8),constrained_layout=True)
plt.plot(sal_prec_month.precipitation,sal_prec_month.Salinity,'.k',ms=8)
plt.ylim([27,35])
plt.xlim([0,9])
plt.grid(ls='--')
#obtain m (slope) and b(intercept) of linear regression line
m, b = np.polyfit(sal_prec_month.precipitation,sal_prec_month.Salinity, 1)
#use red as color for regression line
plt.plot(sal_prec_month.precipitation, (m*sal_prec_month.precipitation+b), color='red',label='Linear regression')
plt.axline((0, 27), slope=1,color='black',ls='--',label='1:1')
plt.text(8, 28, r'r = '+ str(round(sp.stats.pearsonr(sal_prec_month.precipitation,sal_prec_month.Salinity)[0],2)))
plt.legend(bbox_to_anchor=(0, 1, 1, 0), loc="lower left",ncol=3)
plt.gca().set_aspect('equal', 'box')
plt.xlabel('Precipitation (mm)')
plt.ylabel('Salinity')

Text(0, 0.5, 'Salinity')

In [168]:
np.corrcoef(sal_prec_month.precipitation,sal_prec_month.Salinity)

array([[1.        , 0.06740879],
       [0.06740879, 1.        ]])

In [170]:
plt.figure(figsize=(10,8),constrained_layout=True)
plt.plot(sal_prec_month.prec_anom,sal_prec_month.sal_anom,'.k',ms=8)
plt.ylim([-5,5])
plt.xlim([-5,5])
plt.grid(ls='--')
#obtain m (slope) and b(intercept) of linear regression line
m, b = np.polyfit(sal_prec_month.prec_anom,sal_prec_month.sal_anom, 1)
#use red as color for regression line
plt.plot(sal_prec_month.prec_anom, (m*sal_prec_month.prec_anom+b), color='red',label='Linear regression')
plt.axline((-5, -5), slope=1,color='black',ls='--',label='1:1')
plt.text(-4.2, 3, r'r = '+ str(round(sp.stats.pearsonr(sal_prec_month.prec_anom,sal_prec_month.sal_anom)[0],2)))
plt.legend(bbox_to_anchor=(0, 1, 1, 0), loc="lower left",ncol=3)
plt.gca().set_aspect('equal', 'box')
plt.xlabel('Precipitation (mm)')
plt.ylabel('Salinity')

Text(0, 0.5, 'Salinity')

In [171]:
np.corrcoef(sal_prec_month.prec_anom,sal_prec_month.sal_anom)

array([[1.        , 0.00536874],
       [0.00536874, 1.        ]])

## Lagged correlation

In [170]:
# Function to calculate lagged correlation
def lagged_correlation(series1, series2, max_lag):
    corr_values = []
    for lag in range(max_lag + 1):
        correlation = series1.shift(lag).corr(series2)
        corr_values.append(correlation)
    return corr_values

In [172]:
norm_prec = np.linalg.norm(sal_prec_month.prec_anom)
a = sal_prec_month.prec_anom / norm_prec
norm_sal = np.linalg.norm(sal_prec_month.sal_anom)
b = sal_prec_month.sal_anom / norm_sal
c = np.correlate(a, b, mode = 'full')
lags=np.arange(-698, 699)

In [173]:
plt.figure(figsize=(10,8),constrained_layout=True)
plt.axvline(0,color='k')
plt.stem(lags,c)
plt.grid(ls='--')

In [181]:
norm_prec = np.linalg.norm(sal_prec_month.precipitation)
a = sal_prec_month.precipitation / norm_prec
norm_sal = np.linalg.norm(sal_prec_month.Salinity)
b = sal_prec_month.Salinity / norm_sal
c = np.correlate(a, b, mode = 'full')
lags=np.arange(-698, 699)

In [182]:
plt.figure(figsize=(10,8),constrained_layout=True)
plt.axvline(0,color='k')
plt.stem(lags,c)
plt.grid(ls='--')

In [175]:
pv_sal= pd.pivot_table(sal_prec_month, index=sal_prec_month.Date.dt.month, columns=sal_prec_month.Date.dt.year,values='Salinity')

In [177]:
X, Y = np.meshgrid(pv_sal.columns,pv_sal.index)

In [184]:
plt.figure(figsize=(14,8),constrained_layout=True)
img=plt.pcolormesh(X,Y,pv_sal.values,shading='auto', cmap='viridis')
cbar=plt.colorbar(orientation='horizontal',shrink=0.59,pad=0.01,label='Salinity')
plt.ylabel('Months')
plt.xlabel('Years')

Text(0.5, 0, 'Years')

# Heat map

In [185]:
pv_prec= pd.pivot_table(sal_prec_month, index=sal_prec_month.Date.dt.month, columns=sal_prec_month.Date.dt.year,values='precipitation')

In [189]:
plt.figure(figsize=(14,8),constrained_layout=True)
img=plt.pcolormesh(X,Y,pv_prec.values,shading='auto', cmap='Blues')
cbar=plt.colorbar(orientation='horizontal',shrink=0.59,pad=0.01,label='Precipitation (mm)')
plt.ylabel('Months')
plt.xlabel('Years')

Text(0.5, 0, 'Years')

https://download.gebco.net/

In [17]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

Load the netcdf file with xarray package.

In [1]:
import xarray as xr

In [12]:
file='G:/PhD/bathymetry/gebco_2020_n61.0_s49.0_w-5.0_e10.0.nc'
ds_bat = xr.open_dataarray(file,decode_cf=False)
ds_bat = ds_bat.sel(lon=slice(2.5,9.25),lat=slice(53,56))


In [13]:
ds_bat

<xarray.DataArray 'elevation' (lat: 720, lon: 1620)>
[1166400 values with dtype=int16]
Coordinates:
  * lat      (lat) float64 53.0 53.01 53.01 53.01 ... 55.99 55.99 55.99 56.0
  * lon      (lon) float64 2.502 2.506 2.51 2.515 ... 9.235 9.24 9.244 9.248
Attributes:
    standard_name:       height_above_reference_ellipsoid
    long_name:           Elevation relative to sea level
    units:               m
    sdn_parameter_urn:   SDN:P01::BATHHGHT
    sdn_parameter_name:  Sea floor height (above mean sea level) {bathymetric...
    sdn_uom_urn:         SDN:P06::ULAA
    sdn_uom_name:        Metres

## Create a lat/lon grid so we can plot a field.

In [16]:
xx,yy = np.meshgrid(ds_bat.lon, ds_bat.lat)

In [7]:
plt.figure(figsize=(10,8))
ax=plt.axes(projection=ccrs.Mercator(central_longitude=7.1))
img=ds_bat.plot.pcolormesh( shading='auto',
                              cbar_kwargs={'shrink': 0.8,'pad':0.01},extend='neither',transform=ccrs.PlateCarree())
CS=ax.contour(xx,yy,ds_bat,levels=[-30], colors='gray',linestyles=['dashed','solid','-.'],transform=ccrs.PlateCarree())
ax.clabel(CS, inline=True,fontsize=18, fmt='%1.0f',colors='k')
ax.set_xlabel('Longitude (°)')
ax.set_ylabel('Latitude (°)')
ax.plot(7.89,54.19,'*k',markersize=14,markeredgecolor='w',transform=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, facecolor='grey')
ax.coastlines()
ax.set_yticks([53, 54,55,56], crs=ccrs.PlateCarree())
ax.set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
ax.set_yticklabels([53, 54,55,56])
ax.set_xticklabels([3, 6, 9])
plt.text(0.782, 0.89, '(m)')

NameError: name 'ccrs' is not defined